In [14]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import os
import numpy as np
FOLDER_NAME = 'dataset_extracted'


In [4]:
# Create label map
label_map = {}
for (root, folders, files) in os.walk(FOLDER_NAME):
    if root == FOLDER_NAME:
        for index, foldername in enumerate(folders):
            label_map[foldername] = index


print(label_map)


{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [5]:
sequence, label = [], []
target_length = 48
for (root, folders, files) in os.walk(FOLDER_NAME):
    # if root != FOLDER_NAME:
    for filename in files:
        file_path = os.path.join(os.path.relpath(
            root, FOLDER_NAME), filename)
        res = np.load(f'{FOLDER_NAME}/{file_path}')
        for _ in range(target_length-res.shape[0]):
            res = np.vstack((res, res[-1, :]))
        sequence.append(np.array(res))
        label.append(label_map[os.path.basename(root)])

print(np.array(sequence).shape)
print(np.array(label).shape)


(1690, 48, 1662)
(1690,)


In [34]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sequence), keras.utils.to_categorical(
    np.array(label).astype(int), num_classes=26, dtype='float32'), test_size=0.05)

print(X_train.shape, X_test.shape)


(1605, 48, 1662) (85, 48, 1662)
